In [6]:
# Import necessary libraries
from seleniumwire import webdriver  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from bs4 import BeautifulSoup as bs
import csv
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import requests  
from urllib.parse import urlparse
import hashlib  
import json  
import datetime  
import subprocess

# Function to load cookies from a Netscape format cookies.txt file
def load_cookies(browser, file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith('#') and line.strip():
                fields = line.strip().split('\t')
                if len(fields) == 7:
                    cookie = {
                        'domain': fields[0],
                        'flag': fields[1],
                        'path': fields[2],
                        'secure': fields[3],
                        'expiration': fields[4],
                        'name': fields[5],
                        'value': fields[6]
                    }
                    # Adjust domain if it starts with a dot
                    domain = cookie['domain'].lstrip('.')
                    browser.add_cookie({
                        'name': cookie['name'],
                        'value': cookie['value'],
                        'domain': domain,
                        'path': cookie['path'],
                        'expiry': int(cookie['expiration']) if cookie['expiration'] else None
                    })

# Function to get cookies from Selenium Wire browser and convert them to requests-compatible format
def get_requests_cookies(browser):
    selenium_cookies = browser.get_cookies()
    requests_cookies = {}
    for cookie in selenium_cookies:
        requests_cookies[cookie['name']] = cookie['value']
    return requests_cookies

# Function to extract images with "media" in their src, excluding those with 'company-logo'
def extract_images(post_content_container):
    post_images = []
    try:
        post_media_container = post_content_container.find_next("div", {"class": "feed-shared-actor__container"}) if post_content_container else None
        if post_media_container:
            # Extract images
            images = post_media_container.find_all("img", {"class": "feed-shared-image__image"})
            for img in images:
                img_url = img.get('src') or img.get('data-delayed-url')
                if img_url and 'media' in img_url.lower() and 'company-logo' not in img_url.lower():
                    post_images.append(img_url)
    except Exception as e:
        print(f"Error extracting media images: {e}")
    
    # Alternative extraction 
    if not post_images:
        try:
            post_container = post_content_container.find_parent("div", {"class": "occludable-update"}) if post_content_container else None
            if post_container:
                images = post_container.find_all("img")
                for img in images:
                    img_url = img.get('src') or img.get('data-delayed-url')
                    if img_url and 'media' in img_url.lower() and 'company-logo' not in img_url.lower() and 'data:image' not in img_url:
                        post_images.append(img_url)
        except Exception as e:
            print(f"Alternative media image extraction failed: {e}")
    
    # Remove duplicates and return
    return list(set(post_images))

# Function to download images 
def download_images(image_urls, folder_path, post_number):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    for idx, img_url in enumerate(image_urls, start=1):
        try:
            response = requests.get(img_url, timeout=10)
            response.raise_for_status()  # Raise an exception for HTTP errors
            # Extract image extension from URL
            parsed_url = urlparse(img_url)
            _, ext = os.path.splitext(parsed_url.path)
            if not ext:
                ext = '.jpg'  # Default extension if none found
            image_name = f"image_{idx}{ext}"
            image_path = os.path.join(folder_path, image_name)
            with open(image_path, 'wb') as img_file:
                img_file.write(response.content)
            print(f"Downloaded {image_name} for Post {post_number}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download image from {img_url}: {e}")

# Function to extract video URLs via network requests
def extract_m3u8_via_network(requests):
    """
    Looks for an HLS (.m3u8) playlist in the network requests.
    Returns the first .m3u8 URL found, or None if none is found.
    """
    for request in requests:
        if request.response:
            content_type = request.response.headers.get('Content-Type', '').lower()
            if 'application/vnd.apple.mpegurl' in content_type or 'application/x-mpegurl' in content_type:
                # We found an HLS playlist
                return request.url
    return None

def download_hls_video(m3u8_url, folder_path, video_name, requests_cookies):
    """
    Downloads an HLS video from the given m3u8_url using ffmpeg,
    saving it into folder_path as video_name.
    Includes cookies/headers for authentication.
    """
    cookie_string = '; '.join([f'{k}={v}' for k, v in requests_cookies.items()])

    # Example: "Cookie: name1=value1; name2=value2"
    cookie_header = f"Cookie: {cookie_string}"
    referer_header = "Referer: https://www.linkedin.com/"

    output_path = os.path.join(folder_path, video_name)

    # ffmpeg command:
    # -y: overwrite output
    # -headers: supply custom HTTP headers (cookie + referer)
    # -i: input is the .m3u8 URL
    # -c copy: copy the original video/audio data without re-encoding
    command = [
        "ffmpeg",
        "-y",
        "-headers", cookie_header,
        "-headers", referer_header,
        "-i", m3u8_url,
        "-c", "copy",
        output_path
    ]

    print(f"\nAttempting to download HLS video to {output_path}")
    try:
        subprocess.run(command, check=True)
        print(f"Successfully downloaded HLS video as {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"ffmpeg error: {e}")
    except Exception as e:
        print(f"Error downloading HLS video: {e}")

# Scraping function
def scrape_linkedin_posts(max_posts, user_profile_url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in headless mode
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    print("Initializing the Chrome driver with Selenium Wire...")
    browser = webdriver.Chrome(options=chrome_options)

    # Verify that 'requests' attribute exists
    if not hasattr(browser, 'requests'):
        print("Error: 'requests' attribute not found in the WebDriver. Ensure Selenium Wire is correctly installed and imported.")
        browser.quit()
        return

    # Initialize a set to keep track of processed post contents
    processed_contents = set()

    # Tracking: Load processed_contents from a file to persist across runs
    processed_file = 'processed_posts.json'
    if os.path.exists(processed_file):
        with open(processed_file, 'r') as f:
            try:
                processed_contents = set(json.load(f))
                print(f"Loaded {len(processed_contents)} previously processed posts.")
            except json.JSONDecodeError:
                print("Processed posts file is empty or corrupted. Starting fresh.")
                processed_contents = set()

    try:
        # Set the window size
        browser.set_window_size(1920, 1080)

        # Open LinkedIn login page
        print("Opening LinkedIn login page...")
        browser.get('https://www.linkedin.com/')

        # Wait for the page to load
        time.sleep(5)  

        # Load cookies from the file
        print("Loading cookies...")
        load_cookies(browser, 'cookies.txt')  # Ensure 'cookies.txt' is in the same directory
        print("Cookies loaded.")

        # Refresh the page to apply cookies
        browser.refresh()

        # Wait for the main navigation bar to be visible
        print("Waiting for the main navigation bar after applying cookies...")
        try:
            WebDriverWait(browser, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#global-nav .global-nav__me'))
            )
            print("Navigation bar found. Proceeding with scraping...")
        except TimeoutException:
            print("TimeoutException: Navigation bar not found after applying cookies. Check if the cookies are correct.")
            return

        # Navigate to the user's recent activity page
        print(f"Navigating to the user's recent activity page: {user_profile_url}...")
        browser.get(user_profile_url)

        # Wait for the page to load completely
        print("Waiting for the user's recent activity page to load completely...")
        time.sleep(5)  

        # Generate a unique run identifier based on the current datetime
        # run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        # parent_folder = f"Scrape_{run_id}"
        # os.makedirs(parent_folder, exist_ok=True)
        # print(f"Created parent folder: {parent_folder}")
        parent_folder = f"data"

        # Set parameters for scrolling through the page
        SCROLL_PAUSE_TIME = 2  # Time to pause between scrolls
        LOAD_PAUSE_TIME = 10   # Time to wait after scrolling to load new posts

        # Initialize a list to store posts data
        posts_data = []
        post_count = 0

        # Create CSV
        csv_file = f"scraped_linkedin_data.csv"  
        with open(csv_file, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Post Number', 'Content', 'Reactions', 'Comments', 'Media Type', 'Image URL', 'Video URL', 'Article URL'])

        # Get session cookies for requests
        requests_cookies = get_requests_cookies(browser)

        while post_count < max_posts:
            # Parse the page source with BeautifulSoup
            user_page = browser.page_source
            linkedin_soup = bs(user_page.encode("utf-8"), "html.parser")

            # Extract post containers from the HTML
            containers = linkedin_soup.find_all("div", {"class": "occludable-update"})

            # Clear previous network requests
            browser.requests.clear()

            # Main loop to process each container
            print("Processing each container to extract post data...")
            for container in containers:
                if post_count >= max_posts:
                    break

                try:
                    # Extract post content
                    post_content_container = container.find("div", {"class": "update-components-text"})
                    post_content = post_content_container.get_text(separator=' ').strip() if post_content_container else "No content"

                    # Check if the post has no content and skip if so
                    if post_content == "No content":
                        print("Post has no content. Skipping.")
                        continue

                    # Generate a hash of the post content for duplication detection
                    post_hash = hashlib.md5(post_content.encode('utf-8')).hexdigest()

                    if post_hash in processed_contents:
                        print("Duplicate post found based on content. Skipping.")
                        continue  # Skip duplicate posts

                    # Add the hash to the set to mark it as processed
                    processed_contents.add(post_hash)

                except Exception as e:
                    print(f"Error extracting post content: {e}")
                    post_content = "No content"
                    print("Post has no content due to an error. Skipping.")
                    continue

                # Extract Reactions
                try:
                    reactions_button = container.find("li", {"class": "social-details-social-counts__reactions"}).find("button")
                    post_reactions = reactions_button["aria-label"].split(" ")[0].replace(',', '') if reactions_button else "0"
                except:
                    post_reactions = "0"

                # Extract Comments
                try:
                    comments_button = container.find("li", {"class": "social-details-social-counts__comments"}).find("button")
                    post_comments = comments_button["aria-label"].split(" ")[0].replace(',', '') if comments_button else "0"
                except:
                    post_comments = "0"

                # Initialize media content lists
                post_images = []
                post_videos = []
                post_links = []

                # Extract images
                if post_content_container:
                    post_images = extract_images(post_content_container)

                # Extract links
                try:
                    if post_content_container:
                        links = post_content_container.find_all("a", href=True)
                        for link in links:
                            href = link['href']
                            if href.startswith('http'):  # Ensure it's an absolute URL
                                post_links.append(href)
                except Exception as e:
                    print(f"Error extracting links: {e}")

                # Remove duplicates
                post_images = list(set(post_images))
                post_links = list(set(post_links))

                # Determine Media Type (default = 'Text')
                media_type = 'Text'
                videos_str = "No videos"
                images_str = "No images" if not post_images else '; '.join(post_images)
                links_str = "No links" if not post_links else '; '.join(post_links)

                # Increment post count
                post_count += 1
                post_number = post_count

                # Create a folder for the current post
                folder_name = f"Post_{post_number}"
                folder_path = os.path.join(parent_folder, folder_name)
                os.makedirs(folder_path, exist_ok=True)

                # Download images if any
                if post_images:
                    download_images(post_images, folder_path, post_number)

                # Scroll to the post to ensure potential videos are loaded
                try:
                    selenium_post_element = browser.find_element(
                        By.XPATH, f"(//div[contains(@class, 'occludable-update')])[{post_number}]"
                    )
                    browser.execute_script("arguments[0].scrollIntoView();", selenium_post_element)
                    time.sleep(2)  # Wait for dynamic content
                except Exception as e:
                    print(f"Error scrolling to post {post_number}: {e}")

                has_video_element = bool(container.find("video"))  

                # Clear previous requests again, just to capture fresh requests for the current post
                browser.requests.clear()

                if has_video_element:
                    print("Video element detected in the DOM. Waiting for network requests...")
                    # Wait a bit to capture video requests
                    time.sleep(LOAD_PAUSE_TIME)

                    # Extract video URLs from network requests
                    m3u8_url = extract_m3u8_via_network(browser.requests)
                    if m3u8_url:
                        print(f"Found M3U8 link for Post {post_number}: {m3u8_url}")
                        # We'll treat it as a video post
                        media_type = 'Video'
                        videos_str = m3u8_url
                        # Download using ffmpeg
                        download_hls_video(
                            m3u8_url=m3u8_url,
                            folder_path=folder_path,
                            video_name="video_hls.mp4",
                            requests_cookies=requests_cookies
                        )
                    else:
                        print(f"No valid MP4 or M3U8 link found for Post {post_number}.")
                        # This post might not have a playable video
                        media_type = "Text" 
                        videos_str = "No videos"
                else:
                    print(f"No <video> element found for Post {post_number}. Skipping video extraction.")

                # If there's no video, check if it might be an article link or image
                if media_type != 'Video':
                    if any('articleshare-shrink' in img_url for img_url in post_images):
                        media_type = 'Article Link'
                    elif post_images:
                        media_type = 'Image'
                    else:
                        media_type = 'Text'

                # Append post data
                posts_data.append({
                    'Post Number': post_number,
                    'Content': post_content,
                    'Reactions': post_reactions,
                    'Comments': post_comments,
                    'Media Type': media_type,
                    'Images URL': images_str,
                    'Videos URL': videos_str,
                    'Links': links_str,
                })

                print(f"Post {post_count} processed and data saved.")

                # Save the post data to the CSV file immediately
                with open(csv_file, mode='a', encoding='utf-8', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([
                        post_number,
                        post_content,
                        post_reactions,
                        post_comments,
                        media_type,
                        images_str,
                        videos_str,
                        links_str
                    ])

                # Tracking: Save the updated processed_contents to persist across runs
                with open(processed_file, 'w') as f:
                    json.dump(list(processed_contents), f)

            if post_count < max_posts:
                # Scroll down to load more posts
                print("Scrolling down to load more posts...")
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(LOAD_PAUSE_TIME)
            else:
                break

        print("Finished scrolling and processing posts.")
        print(f"Data exported to {csv_file}")

    finally:
        # Close the browser
        browser.quit()

# Function to handle user input and initiate scraping
def start_scraping(button):
    with output:
        clear_output()
        try:
            max_posts = int(post_count_widget.value)
            user_profile_url = profile_url_widget.value.strip()
            if not user_profile_url:
                print("Please enter a valid LinkedIn profile URL.")
                return
            print(f"Starting scraping for {max_posts} posts from {user_profile_url}...")
            scrape_linkedin_posts(max_posts, user_profile_url)
        except ValueError:
            print("Please enter a valid number for the number of posts.")

# Create interactive widgets
post_count_widget = widgets.BoundedIntText(
    value=10,
    min=1,
    max=1000,
    step=1,
    description='Number of Posts:',
    disabled=False
)

profile_url_widget = widgets.Textarea(
    value='https://www.linkedin.com/school/nyushanghai/posts/?feedView=all',
    placeholder='Enter LinkedIn Profile URL',
    description='Profile URL:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='60px')  # Adjust size as needed
)

submit_button = widgets.Button(
    description='Start Scraping',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to start scraping',
    icon='check'
)

output = widgets.Output()

submit_button.on_click(start_scraping)

# Display the widgets
display(post_count_widget, profile_url_widget, submit_button, output)

BoundedIntText(value=10, description='Number of Posts:', max=1000, min=1)

Textarea(value='https://www.linkedin.com/school/nyushanghai/posts/?feedView=all', description='Profile URL:', …

Button(button_style='success', description='Start Scraping', icon='check', style=ButtonStyle(), tooltip='Click…

Output()